# Flink + Iceberg Query Examples

This notebook demonstrates querying Iceberg tables using PyFlink in the **flink-dev Docker environment**.

## Environment Setup
- **Container**: `flink-dev` (from docker-compose.yml)
- **Flink Cluster**: Running inside flink-dev container
- **Jupyter Lab**: http://localhost:8889
- **Flink Web UI**: http://localhost:8082
- **JAR Configuration**: Minimal setup with iceberg-flink-runtime and hadoop-common

## Prerequisites
1. Start the development stack: `docker-compose up flink-dev`
2. Ensure Iceberg tables exist (run notebooks 1-7 first)
3. Access this notebook through Jupyter Lab in the container


In [7]:
# Environment Verification - flink-dev container
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.datastream import StreamExecutionEnvironment
import os
import sys
import glob

print("=== Flink Development Environment ===")
print(f"Container: flink-dev")
print(f"PyFlink imported successfully!")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")
print(f"Flink home: {os.environ.get('FLINK_HOME', 'Not set')}")


=== Flink Development Environment ===
Container: flink-dev
PyFlink imported successfully!
Python version: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
Working directory: /opt/workspace
Flink home: /opt/flink


In [8]:
# Verify JAR configuration in flink-dev environment
jar_paths = ["/opt/flink/lib/*.jar"]

print("=== JAR Configuration Verification ===")
print("Location: /opt/flink/lib/ (inside flink-dev container)")
print("=" * 60)

iceberg_jars = []
hadoop_jars = []
other_jars = []

for path_pattern in jar_paths:
    jars = glob.glob(path_pattern)
    if jars:
        for jar in sorted(jars):
            jar_name = os.path.basename(jar)
            jar_size = os.path.getsize(jar) / (1024 * 1024)  # MB
            
            if "iceberg" in jar_name.lower():
                iceberg_jars.append((jar_name, jar_size))
            elif "hadoop" in jar_name.lower():
                hadoop_jars.append((jar_name, jar_size))
            else:
                other_jars.append((jar_name, jar_size))

print("Iceberg JARs:")
for jar_name, size in iceberg_jars:
    print(f"  ✓ {jar_name} ({size:.1f} MB)")

print("\nHadoop JARs:")
for jar_name, size in hadoop_jars:
    print(f"  ✓ {jar_name} ({size:.1f} MB)")

print(f"\nOther Flink JARs: {len(other_jars)} files")
print(f"Total JAR files: {len(iceberg_jars) + len(hadoop_jars) + len(other_jars)}")


=== JAR Configuration Verification ===
Location: /opt/flink/lib/ (inside flink-dev container)
Iceberg JARs:
  ✓ iceberg-aws-bundle-1.9.1.jar (57.6 MB)
  ✓ iceberg-flink-runtime-1.20-1.9.1.jar (34.7 MB)

Hadoop JARs:
  ✓ flink-s3-fs-hadoop-1.20.0.jar (30.1 MB)
  ✓ flink-shaded-hadoop-2-uber-2.8.3-10.0.jar (41.3 MB)
  ✓ hadoop-common-3.3.4.jar (4.3 MB)
  ✓ hadoop-hdfs-client-3.3.4.jar (5.2 MB)

Other Flink JARs: 13 files
Total JAR files: 19


In [9]:
# Initialize Flink Table Environment for batch processing
print("=== Flink Environment Initialization ===")

# Create batch environment (optimal for Iceberg queries)
settings = EnvironmentSettings.new_instance().in_batch_mode().build()
table_env = TableEnvironment.create(settings)

print("Flink Table Environment created:")
print(f"  Mode: Batch (optimized for Iceberg)")
print(f"  Environment: flink-dev container")
print(f"  Ready for Iceberg operations")

=== Flink Environment Initialization ===
Flink Table Environment created:
  Mode: Batch (optimized for Iceberg)
  Environment: flink-dev container
  Ready for Iceberg operations


In [10]:
# Verify available catalogs in flink-dev environment
print("=== Available Catalogs ===")
table_env.execute_sql("SHOW CATALOGS").print()


=== Available Catalogs ===
+-----------------+
|    catalog name |
+-----------------+
| default_catalog |
+-----------------+
1 row in set


In [11]:
# Configure Iceberg catalog connection
print("=== Iceberg Catalog Configuration ===")
print("Connecting to Iceberg REST catalog...")

# Create Iceberg catalog with connection to services in docker-compose stack
catalog_sql = """
CREATE CATALOG IF NOT EXISTS iceberg_catalog WITH (
    'type' = 'iceberg',
    'catalog-type'='rest',
    'uri' = 'http://iceberg-rest:8181',
    'warehouse' = 's3://warehouse/',
    'io-impl' = 'org.apache.iceberg.aws.s3.S3FileIO',
    's3.endpoint' = 'http://minio:9000',
    's3.access-key-id' = 'admin',
    's3.secret-access-key' = 'password',
    's3.path-style-access' = 'true'
)
"""

table_env.execute_sql(catalog_sql)
table_env.use_catalog("iceberg_catalog")

print("✓ Iceberg catalog configured successfully")
print("✓ Connected to REST catalog at http://iceberg-rest:8181")
print("✓ Using MinIO storage at http://minio:9000")

=== Iceberg Catalog Configuration ===
Connecting to Iceberg REST catalog...
✓ Iceberg catalog configured successfully
✓ Connected to REST catalog at http://iceberg-rest:8181
✓ Using MinIO storage at http://minio:9000


In [12]:
# Access the play_iceberg namespace and list available tables
print("=== Accessing Iceberg Tables ===")
print("Switching to play_iceberg namespace...")

table_env.execute_sql("USE iceberg_catalog.play_iceberg")

print("Available tables in play_iceberg namespace:")
table_env.execute_sql("SHOW TABLES").print()


=== Accessing Iceberg Tables ===
Switching to play_iceberg namespace...
Available tables in play_iceberg namespace:
+------------+
| table name |
+------------+
|      users |
+------------+
1 row in set


In [13]:
# Query Iceberg table using Flink SQL
print("=== Querying Iceberg Table ===")
print("Executing: SELECT * FROM users LIMIT 10")
print()

# Execute query and display results
table_env.execute_sql("SELECT * FROM users LIMIT 10").print()

print()
print("✓ Successfully queried Iceberg table using PyFlink")
print("✓ Data retrieved from MinIO storage via Iceberg REST catalog")
print("✓ flink-dev environment working correctly")

=== Querying Iceberg Table ===
Executing: SELECT * FROM users LIMIT 10

Empty set

✓ Successfully queried Iceberg table using PyFlink
✓ Data retrieved from MinIO storage via Iceberg REST catalog
✓ flink-dev environment working correctly


In [14]:
# Additional query examples
print("=== Additional Query Examples ===")

# Count total records
print("1. Count total users:")
table_env.execute_sql("SELECT COUNT(*) as total_users FROM users").print()

# Active vs inactive users
print("\n2. Active vs Inactive users:")
table_env.execute_sql("""
    SELECT is_active, COUNT(*) as user_count 
    FROM users 
    GROUP BY is_active
""").print()

# Recent users (latest 5)
print("\n3. Latest 5 users by updated timestamp:")
table_env.execute_sql("""
    SELECT user_id, username, updated_at 
    FROM users 
    ORDER BY updated_at DESC 
    LIMIT 5
""").print()

print("\n=== flink-dev Environment Validation Complete ===")
print("✓ All Iceberg operations successful")
print("✓ PyFlink integration working")
print("✓ Docker services connected properly")


=== Additional Query Examples ===
1. Count total users:
2025-07-01 14:13:36,501 INFO  org.apache.hadoop.io.compress.CodecPool                      [] - Got brand-new decompressor [.zstd]
+----------------------+
|          total_users |
+----------------------+
|                    5 |
+----------------------+
1 row in set

2. Active vs Inactive users:
2025-07-01 14:13:39,216 INFO  org.apache.hadoop.io.compress.CodecPool                      [] - Got brand-new decompressor [.zstd]
+-----------+----------------------+
| is_active |           user_count |
+-----------+----------------------+
|      TRUE |                    4 |
|     FALSE |                    1 |
+-----------+----------------------+
2 rows in set

3. Latest 5 users by updated timestamp:
2025-07-01 14:13:40,982 INFO  org.apache.hadoop.io.compress.CodecPool                      [] - Got brand-new decompressor [.zstd]
+----------------------+--------------------------------+----------------------------+
|              user